# Vision Transformer on MedMNIST 2D Multi-Class

[MedMNIST v2](https://medmnist.com/) is a collection of biomedical images.
There are 8 datasets with 2D images for a multi-class classification task.
The authors of the dataset collection report baseline performances with ResNets,
and with the AutoML solutions auto-sklearn, AutoKeras, and Google AutoML Vision.

Using a pre-trained [Vision Transformer model](https://huggingface.co/google/vit-base-patch16-224), and fine-tuning it for each task,
we are able to outperform almost all of those baselines.
This notebook contains the full code to run the Vision Transformer experiment.

Sources:
* Dataset: https://medmnist.com/
* Model: https://huggingface.co/google/vit-base-patch16-224
* Fine tuning: https://github.com/NielsRogge/Transformers-Tutorials/blob/master/VisionTransformer/Fine_tuning_the_Vision_Transformer_on_CIFAR_10_with_the_%F0%9F%A4%97_Trainer.ipynb

### Imports

In [1]:
import numpy as np
import torch

from datasets import load_dataset
from transformers import ViTImageProcessor, ViTForImageClassification, TrainingArguments, Trainer, EarlyStoppingCallback

from torchvision.transforms import (CenterCrop, 
                                    Compose, 
                                    Normalize, 
                                    RandomHorizontalFlip,
                                    RandomResizedCrop, 
                                    Resize, 
                                    ToTensor)

from sklearn.metrics import roc_auc_score
import evaluate

### Loading data and transforms

In [2]:
modelname = "google/vit-base-patch16-224"

processor = ViTImageProcessor.from_pretrained(modelname)

image_mean, image_std = processor.image_mean, processor.image_std
size = processor.size["height"]

normalize = Normalize(mean=image_mean, std=image_std)
_train_transforms = Compose(
        [
            RandomResizedCrop(size),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )

_val_transforms = Compose(
        [
            Resize(size),
            CenterCrop(size),
            ToTensor(),
            normalize,
        ]
    )

def train_transforms(examples):
    examples['pixel_values'] = [_train_transforms(image.convert("RGB")) for image in examples['image']]
    return examples

def val_transforms(examples):
    examples['pixel_values'] = [_val_transforms(image.convert("RGB")) for image in examples['image']]
    return examples

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [3]:
# list of datasets to use
dataset_2d_list = ['pathmnist', 'dermamnist', 'octmnist', 'bloodmnist',
                   'tissuemnist', 'organamnist', 'organcmnist', 'organsmnist']

In [4]:
def load_dataset_2d(dataset_name):
    '''
    Loads dataset with the given name and returns train, val, and test datasets,
    and dictionaries to convert between labels and ids.
    '''

    dataset = load_dataset("albertvillanova/medmnist-v2", dataset_name)

    train_ds = dataset['train']
    val_ds = dataset['validation']
    test_ds = dataset['test']

    id2label = {id:label for id, label in enumerate(train_ds.features['label'].names)}
    label2id = {label:id for id,label in id2label.items()}
    id2label

    # Set the transforms
    train_ds.set_transform(train_transforms)
    val_ds.set_transform(val_transforms)
    test_ds.set_transform(val_transforms)

    return train_ds, val_ds, test_ds, id2label, label2id

### Set up training and evaluation

In [5]:
args = TrainingArguments(
    f"medmnist-vit-1",
    save_strategy = "epoch",
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 60,
    per_device_eval_batch_size = 40,
    num_train_epochs = 50,
    weight_decay = 0.01,
    load_best_model_at_end = True,
    metric_for_best_model = "roc_auc",
    remove_unused_columns = False,
)

In [6]:
def compute_metrics(eval_preds):
    '''
    Calculate accuracy and roc_auc.
    '''

    # accuracy
    metric_accuracy = evaluate.load("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric_accuracy.compute(predictions=predictions, references=labels)['accuracy']

    # calculate roc_auc the same way it is calculated for the MedMNIST benchmark:
    # https://github.com/MedMNIST/MedMNIST/blob/main/medmnist/evaluator.py
    auc = 0
    for i in range(logits.shape[1]):
        y_true_binary = (labels == i).astype(float)
        y_score_binary = logits[:, i]
        auc += roc_auc_score(y_true_binary, y_score_binary)
    roc_auc = auc / logits.shape[1]
    
    return {'accuracy': accuracy, 'roc_auc': roc_auc}

In [7]:
def train_and_evaluate(train_ds, val_ds, test_ds, id2label, label2id):
    '''
    Train the model and evaluate on the test set.
    '''

    model = ViTForImageClassification.from_pretrained(modelname, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)

    trainer = Trainer(
        model,
        args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        data_collator=collate_fn,
        compute_metrics=compute_metrics,
        tokenizer=processor,
        callbacks = [EarlyStoppingCallback(early_stopping_patience=5)],
    )

    trainer.train()

    # evaluate
    outputs = trainer.predict(test_ds)
    print(outputs.metrics)

### Train and evaluate each dataset

In [8]:
for dataset_name in dataset_2d_list:
    print(dataset_name)
    train_ds, val_ds, test_ds, id2label, label2id = load_dataset_2d(dataset_name)
    train_and_evaluate(train_ds, val_ds, test_ds, id2label, label2id)

pathmnist


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([9]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([9, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Roc Auc
1,0.162400,0.070312,0.976309,0.999329
2,0.104500,0.044090,0.986505,0.999695
3,0.081600,0.025287,0.991204,0.999861
4,0.073600,0.019002,0.994002,0.999924
5,0.058700,0.017753,0.994302,0.999915
6,0.053400,0.014458,0.995602,0.999942
7,0.049600,0.011532,0.996401,0.999974
8,0.048100,0.006128,0.998301,0.999989
9,0.039900,0.012785,0.996202,0.999962
10,0.039500,0.007764,0.997401,0.999991


{'test_loss': 0.42436450719833374, 'test_accuracy': 0.9143454038997214, 'test_roc_auc': 0.9874672401641509, 'test_runtime': 15.3634, 'test_samples_per_second': 467.343, 'test_steps_per_second': 11.716}
dermamnist


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([7]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch,Training Loss,Validation Loss,Accuracy,Roc Auc
1,No log,0.243618,0.939799,0.996051
2,No log,0.144212,0.960284,0.998926
3,0.722500,0.103046,0.974080,0.999595
4,0.722500,0.066718,0.983696,0.999655
5,0.388000,0.098959,0.973662,0.999471
6,0.388000,0.071873,0.982023,0.999540
7,0.324400,0.085149,0.976171,0.999523
8,0.324400,0.072209,0.982441,0.999540
9,0.324400,0.073911,0.978679,0.999723
10,0.279800,0.062673,0.984114,0.999622


{'test_loss': 0.2132023572921753, 'test_accuracy': 0.9327527818093856, 'test_roc_auc': 0.9930285838724114, 'test_runtime': 17.1878, 'test_samples_per_second': 481.039, 'test_steps_per_second': 12.043}
organsmnist


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([11]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([11, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Roc Auc
1,No log,0.463623,0.838091,0.985340
2,No log,0.344777,0.868679,0.989933
3,0.914100,0.296895,0.878874,0.991234
4,0.914100,0.257948,0.886215,0.991494
5,0.574900,0.267272,0.886623,0.991809
6,0.574900,0.248179,0.890701,0.992434
7,0.497000,0.206471,0.909462,0.993265
8,0.497000,0.212156,0.909054,0.992954
9,0.434700,0.206258,0.911909,0.993092
10,0.434700,0.207539,0.910685,0.993504


{'test_loss': 0.5182576775550842, 'test_accuracy': 0.8277268093781855, 'test_roc_auc': 0.9812998686754533, 'test_runtime': 18.6595, 'test_samples_per_second': 473.163, 'test_steps_per_second': 11.844}
